In [1]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostClassifier
%matplotlib inline


le = preprocessing.LabelEncoder()

### vuelvo a traer mi train con los embeddings de nlp

In [2]:
train = pd.read_csv('csv/train_2_KNN_embeddings.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      7613 non-null   int64  
 1   id              7613 non-null   int64  
 2   keyword         7552 non-null   object 
 3   location        5080 non-null   object 
 4   text            7613 non-null   object 
 5   target          7613 non-null   int64  
 6   word2vec_score  7613 non-null   float64
 7   elmo_score      7613 non-null   float64
 8   glove_score     7613 non-null   float64
 9   bert_score      7613 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 594.9+ KB


In [3]:
train.head(10)

,Unnamed: 0,id,keyword,location,text,target,word2vec_score,elmo_score,glove_score,bert_score
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.556296,0.522046,0.587237,0.567960
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.674412,0.538970,0.676435,0.548946
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.725413,0.583167,0.668918,0.589782
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.758587,0.650540,0.656915,0.602543
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.598320,0.570466,0.573899,0.531318
5,5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,0.715882,0.652919,0.756050,0.708653
6,6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,0.701084,0.754674,0.785661,0.753023
7,7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,0.418606,0.347374,0.567888,0.468912
8,8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,0.653519,0.509503,0.648205,0.668390
9,9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,0.514508,0.532526,0.579315,0.458969


### pero en este caso agrego una dimension  extra que es un  encoding de la ubicacion.

In [4]:
train ['label_geo'] = le.fit_transform( train['location'].to_list() )

In [5]:
train.head()

,Unnamed: 0,id,keyword,location,text,target,word2vec_score,elmo_score,glove_score,bert_score,label_geo
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.556296,0.522046,0.587237,0.567960,3127
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.674412,0.538970,0.676435,0.548946,3127
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.725413,0.583167,0.668918,0.589782,3127
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.758587,0.650540,0.656915,0.602543,3127
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.598320,0.570466,0.573899,0.531318,3127


In [6]:
x = train[ ['word2vec_score', 'elmo_score', 'glove_score', 'bert_score', 'label_geo'] ]
y = train['target']

In [7]:
x.sample(3)

,word2vec_score,elmo_score,glove_score,bert_score,label_geo
4446,0.571776,0.681571,0.606632,0.691649,369
1913,0.256132,0.311342,0.273816,0.254016,3127
645,0.199526,0.223170,0.202479,0.279815,707


In [8]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [9]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [10]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

Accuracy Score : 0.7556917688266199
Precision Score : 0.7412663755458515
Recall Score : 0.679
F1 Score : 0.708768267223382


### el resultado no es muy bueno